# Shortest distances based on walking DB
This notebook is used to efficiently load in data from the postgres DB, connect it with data from the API to effectively calculate the shortest walking distances from all address points to the nearest container for all fractions. In a more general way, there are a few essential steps:
* Connect to postgres DB and make it easy to send queries
* Load all POI clusters from the postgres DB
* Load all garbage clusters from the data.amsterdam API
* Connect both databases on their mutual coordinates
* Load distances matrix from postgres DB
* Join distances matrix with garbage cluster data
* Join distances matrix with POI clusters


In [1]:
# Load all necessary modules for working

from sqlalchemy import create_engine
import requests
from getpass import getpass
import pandas as pd
import numpy as np
import psycopg2
from shapely.geometry import Point, Polygon

### Connect to Postgres DB
First step is to fill in all necessary parameters to make a connection to the DB.
These functions allow for easy implementation

In [2]:
# Connect to Postgres database
# source: https://blog.panoply.io/connecting-jupyter-notebook-with-postgresql-for-python-data-analysis

# Postgres username, password, and database name
POSTGRES_ADDRESS = '10.243.25.5'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'heijne029'
POSTGRES_PASSWORD = getpass(prompt='Password: ')
POSTGRES_DBNAME = 'analyse_ruimte'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME,
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

Password: ········


In [3]:
def get_dataframe(q):
    # Create the connection
    cnx = create_engine(postgres_str)
    # Query to send to database
    query = q
    return pd.read_sql_query(query, cnx)

### Helper functions

In [4]:
# def rest_in_list(fraction_list):
#     if type(fraction_list) == list:
#         for fraction in fraction_list:
#             if fraction[:4] == "Rest":
#                 return int(fraction[5:])
#     else:
#         return 0
    

# def plastic_in_list(fraction_list):
#     if type(fraction_list) == list:
#         for fraction in fraction_list:
#             if fraction[:7] == "Plastic":
#                 return int(fraction[8:])
#     else:
#         return 0

# def papier_in_list(fraction_list):
#     if type(fraction_list) == list:
#         for fraction in fraction_list:
#             if fraction[:6] == "Papier":
#                 return int(fraction[7:])
#     else:
#         return 0

# def glas_in_list(fraction_list):
#     if type(fraction_list) == list:
#         for fraction in fraction_list:
#             if fraction[:4] == "Glas":
#                 return int(fraction[5:])
#     else:
#         return 0

# def textiel_in_list(fraction_list):
#     if type(fraction_list) == list:
#         for fraction in fraction_list:
#             if fraction[:7] == "Textiel":
#                 return int(fraction[8:])
#     else:
#         return 0

def containers_per_cluster(cluster_list):
    rest = 0
    plastic = 0
    papier = 0
    glas = 0
    textiel = 0
    try:
        for i in cluster_list:
            if i.startswith("Rest:"):
                rest = int((i.split(':')[1]))
            if i.startswith("Plastic:"):
                plastic = int((i.split(':')[1]))
            if i.startswith("Papier:"):
                papier = int((i.split(':')[1]))
            if i.startswith("Glas:"):
                glas = int((i.split(':')[1]))
            if i.startswith("Textiel:"):
                textiel = int(i.split(':')[1])
    except:
        pass
    return rest, plastic, papier, glas, textiel, sum([rest, plastic, papier, glas, textiel])

In [5]:
def load_api_data(prnt=False):
    x_coordinates = []
    y_coordinates = []
    aantal = []
    volumes = []
    adresses = []

    link = 'https://api.data.amsterdam.nl/vsd/afvalclusters'

    while link != None:
        if prnt:
            print(link)
        response = requests.get(link)
        output = response.json()
        for result in output['results']:
            if result['cluster_datum_einde_cluster'] == None: #Als het cluster nog actief is
                x_coordinates.append(str(result['cluster_geometrie']['coordinates'][0]))
                y_coordinates.append(str(result['cluster_geometrie']['coordinates'][1]))
                aantal.append(result['cluster_fractie_aantal'])
                volumes.append(result['cluster_fractie_volume'])
                adresses.append(result['bag_adres_openbare_ruimte_naam'])
        try:
            link = output['_links']['next']['href']
        except:
            link = None

    df_clusters = pd.DataFrame([x_coordinates, y_coordinates, aantal, volumes, adresses]).T
    df_clusters = df_clusters.rename(columns={0: 'cluster_x', 1:'cluster_y', 2:'aantal_per_fractie', 3:'volume_per_fractie', 4: 'street_name'})
    df_clusters['cluster_x'] = df_clusters['cluster_x'].astype('float').round(0).astype('int')
    df_clusters['cluster_y'] = df_clusters['cluster_y'].astype('float').round(0).astype('int')
    return df_clusters

### Load in DB and API
the PostgresDB is loaded in to retrieve all information.

Next, the API is used to load in all clusters

In [6]:
df_afstandn3 = get_dataframe("""SELECT *
                                FROM proj_afval_netwerk.afv_rel_nodes_poi
                                """)

In [7]:
df_clusters = load_api_data()

### Retrieve amount of containers per fraction for all clusters
This creates a dataframe with the distance between an address POI and all nearby garbage clusters. It also holds the amount of containers per fraction for this garbage cluster

In [8]:
# total_join['rest'] = total_join['aantal_per_fractie'].apply(lambda x: rest_in_list(x))
# total_join['plastic'] = total_join['aantal_per_fractie'].apply(lambda x: plastic_in_list(x))
# total_join['glas'] = total_join['aantal_per_fractie'].apply(lambda x: glas_in_list(x))
# total_join['papier'] = total_join['aantal_per_fractie'].apply(lambda x: papier_in_list(x))
# total_join['textiel'] = total_join['aantal_per_fractie'].apply(lambda x: textiel_in_list(x))
# total_join['totaal'] = total_join['rest'] + total_join['plastic'] + total_join['glas'] + total_join['papier'] + total_join['textiel']
df_clusters['rest'], df_clusters['plastic'], df_clusters['papier'], df_clusters['glas'], df_clusters['textiel'], df_clusters['totaal'] = zip(*df_clusters['aantal_per_fractie'].apply(lambda x: containers_per_cluster(x)))

### Join DB and API data
Next step is to join API and DB data.
Herefore, the data is joined on coordinates.
When there was no match in that, the remaining clusters were tried to be matched on more flexible norms with margins of 25 meters. This greatly reduces missing values.
The initial matches are stored in result

In [9]:
def pois_for_clusters(tst=df_afstandn3, df_clusters=df_clusters):
    tst['woning'] = tst['bk_afv_rel_nodes_poi'].str.split('~')
    tst['cluster_x'] = tst['woning'].apply(lambda x: x[0]).astype('float').round(0).astype('int')
    tst['cluster_y'] = tst['woning'].apply(lambda x: x[1]).astype('float').round(0).astype('int')
    tst['type'] = tst['woning'].apply(lambda x: x[2])
    db_clusters = tst[tst['type'] == 'afval_cluster']
    
    joined = db_clusters.set_index(['cluster_x', 'cluster_y']).join(df_clusters.set_index(['cluster_x', 'cluster_y']), how='outer').reset_index()
    
    df_clusters_open = joined[joined['s1_afv_rel_nodes_poi'].isna()].reset_index()
    db_clusters_open = joined[joined['aantal_per_fractie'].isna()].reset_index()
    
    
    return joined.dropna(), db_clusters_open, df_clusters_open

In [10]:
result, db_clusters_open, df_clusters_open = pois_for_clusters()
result.shape, db_clusters_open.shape, df_clusters_open.shape

((5578, 22), (96, 23), (106, 23))

In [11]:
 def fix_remaining_options(i, db_clusters_open, df_clusters_open, margin=10):
    test_option = db_clusters_open.iloc[i]
    x = float(test_option['cluster_x'])
    y = float(test_option['cluster_y'])
#     print(x,y)
    square = Polygon([(x-margin, y-margin), (x-margin, y+margin), (x+margin, y+margin), (x+margin, y-margin)])
    df_clusters['point'] = df_clusters.apply(lambda row: Point(row['cluster_x'], row['cluster_y']),axis=1)
    df_clusters['fit'] = df_clusters['point'].apply(lambda point: point.within(square))
    try:
        to_return = df_clusters[df_clusters['fit']].iloc[0]
        return to_return['aantal_per_fractie'], to_return['volume_per_fractie'], to_return['street_name']
    except:
        return None, None, None
    
def fix_remaining_into_frame(db_clusters_open=db_clusters_open, df_clusters_open=df_clusters_open, margin=10):
    apf_list = []
    vpf_list = []
    str_list = []

    for i in range(db_clusters_open.shape[0]):
        tmp = fix_remaining_options(i, db_clusters_open, df_clusters_open, margin=margin)
        apf_list.append(tmp[0])
        vpf_list.append(tmp[1])
        str_list.append(tmp[2])

    db_clusters_open['aantal_per_fractie'] = apf_list
    db_clusters_open['volume_per_fractie'] = vpf_list
    db_clusters_open['street_name'] = str_list
    return db_clusters_open

In [12]:
probeersel = fix_remaining_into_frame(margin=30).drop('index', axis=1)
probeersel['rest'], probeersel['plastic'], probeersel['papier'], probeersel['glas'], probeersel['textiel'], probeersel['totaal'] = zip(*probeersel['aantal_per_fractie'].apply(lambda x: containers_per_cluster(x)))
new_result = result.append([probeersel],ignore_index=True)

### Load in distances matrix from postgres DB
The afv_poi_afstand is used toa find all distances. The pois here can be used later on to couple them to the objects they actually belong to. First step is the loading of the distances however.

In [13]:
df_afstandn2 = get_dataframe("""SELECT * 
                                FROM proj_afval_netwerk.afv_poi_afstand
                                WHERE afstand < 1000""")

### Join distances matrix with POI clusters

In [14]:
tst = df_afstandn2.set_index('naar_s1_afv_nodes').join(df_afstandn3.set_index('s1_afv_nodes')).reset_index().rename(columns={'index': 'naar_s1_afv_nodes'})

### Join distances matrix with enriched garbage clusters
This is the last step of the general repetitive part. This makes this point a great endpoint to dump the outcome into a .csv to be used in other notebooks. This makes the next notebooks also a bit more compact and easy to read

In [15]:
total_join = tst.set_index('van_s1_afv_nodes').join(new_result.set_index('s1_afv_nodes'), rsuffix='_right', how='left')

In [16]:
total_join = total_join.fillna(method='ffill', limit=1)
total_join = total_join.reset_index().rename(columns={'index': 'van_s1_afv_nodes'}).sort_values(by='afstand')

In [17]:
to_csv = False
if to_csv:
    total_join[['van_s1_afv_nodes', 'naar_s1_afv_nodes', 'afstand', 'bk_afv_rel_nodes_poi', 'cluster_x', 'cluster_y', 'volume_per_fractie', 'rest', 'plastic', 'papier', 'glas', 'textiel', 'totaal']].to_csv('cluster_join.csv', compression='gzip')

### Calculate shortest distance to all fractions
This part is on finding the shortest distance from every house to its nearest container for all fractions

In [18]:
# def shortest_distance(house_poi, fractie, df = total_join):
#     """
#     Search method to find nearest container for a certain house of a fraction.
#     It takes as input the house_poi_number, fraction and a dataframe to look in
#     Output is the distance to the nearest container of that fraction
#     """
#     possibilities = df[df['naar_s1_afv_nodes'] == house_poi].sort_values(by='afstand', ascending=True)[[fractie, 'afstand']]
# #     print(possibilities)
    
#     for i in range(possibilities.shape[0]):
#         if possibilities.iloc[i][fractie] > 0:
#             return possibilities.iloc[i]['afstand']
#     return np.nan

# def shortest_distance_container(x, fractie, df = total_join):
#     """
#     Search method to find nearest container for a certain house of a fraction.
#     It takes as input the house_poi_number, fraction and a dataframe to look in
#     Output is the distance to the nearest container of that fraction
#     """
#     possibilities = df[df['naar_s1_afv_nodes'] == x['naar_s1_afv_nodes']].sort_values(by='afstand', ascending=True)[[fractie, 'afstand', 'van_s1_afv_nodes']]
    
#     for i in range(possibilities.shape[0]):
#         if possibilities.iloc[i][fractie] > 0:
# #             print(possibilities.iloc[i]['afstand'], possibilities.iloc[i]['s1_afv_poi_van'])
#             return possibilities.iloc[i]['afstand'], possibilities.iloc[i]['naar_s1_afv_nodes']
# #     print('a')
#     return np.nan, np.nan

def shortest_distance(house_poi, fractie):
    try:
        possibilities = total_join[(total_join['naar_s1_afv_nodes'] == house_poi) & (total_join[fractie] > 0)].sort_values(by='afstand')
#         print(possibilities.iloc[0]['afstand'])
        return possibilities.iloc[0]['afstand'], possibilities.iloc[0]['van_s1_afv_nodes']
    except:
        print('nothing')
        return np.nan, np.nan

In [19]:
all_households1 = total_join[['naar_s1_afv_nodes', 'bk_afv_rel_nodes_poi']].drop_duplicates()
all_households1['woning'] = all_households1['bk_afv_rel_nodes_poi'].str.split('~')
all_households1['woning_x'] = all_households1['woning'].apply(lambda x: x[0])
all_households1['woning_y'] = all_households1['woning'].apply(lambda x: x[1])
all_households1 = all_households1.drop('bk_afv_rel_nodes_poi', axis=1)

In [20]:
slow = False
if slow:
    all_households = all_households1[:1000]
    for i in ['rest', 'papier', 'plastic', 'textiel', 'glas']:
        print('****************************************************************')
        afstand = 'afstand_' + i
        poi = i + '_poi'
        all_households[afstand], all_households[poi] = zip(*all_households.apply(lambda x: shortest_distance(x['naar_s1_afv_nodes'], i), axis=1))    

In [21]:
shortest_rest = total_join[total_join['rest'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_rest', 'afstand': 'rest_afstand'})
shortest_plastic = total_join[total_join['plastic'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_plastic', 'afstand': 'plastic_afstand'})
shortest_papier = total_join[total_join['papier'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_papier', 'afstand': 'papier_afstand'})
shortest_glas = total_join[total_join['glas'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_glas', 'afstand': 'glas_afstand'})
shortest_textiel = total_join[total_join['textiel'] > 0].groupby('naar_s1_afv_nodes').first()[['van_s1_afv_nodes', 'afstand']].rename(columns={'van_s1_afv_nodes': 'poi_textiel', 'afstand': 'textiel_afstand'})

all_households = all_households1.set_index('naar_s1_afv_nodes').join([shortest_rest, shortest_plastic, shortest_papier, shortest_glas, shortest_textiel], how='left')
all_households.isna().sum()

woning                 0
woning_x               0
woning_y               0
poi_rest            3210
rest_afstand        3210
poi_plastic         1695
plastic_afstand     1695
poi_papier          1327
papier_afstand      1327
poi_glas            1381
glas_afstand        1381
poi_textiel        11050
textiel_afstand    11050
dtype: int64

### Filter results on location and general waste garbage plan
Some of the households are not located within the areas where general waste is collected. Another column should be added to all_households to obtain whether or not it is part of this area

In [22]:
import geopandas as gpd
import shapely

source = gpd.read_file('../data/Inzameling_huisvuil_100220.shp')
source = source[source['aanbiedwij'] == 'Breng uw restafval  naar een container voor restafval.']
polygons = list(source.geometry)

def adress_in_service_area(x, y, polygon_list):
    """
    function to see whether a certain household is within the service area of rest
    Input is x and y coordinates of a house and a list of polygons of service area.
    Returns boolean
    """
    point = shapely.geometry.Point(float(x),float(y))
    for polygon in polygon_list:
        if polygon.contains(point):
            return True
    return False

In [30]:
all_households['uses_container'] = all_households.apply(lambda row: adress_in_service_area(row['woning_x'], row['woning_y'], polygons), axis=1)
good_result = all_households[all_households['uses_container']]

### Calculate Score
With this information available, it is possible to calculate the average weighted walking distance as part of the total score function

In [24]:
def calculate_weighted_distance(good_result):
    rest_mean = good_result['rest_afstand'].mean()
    papier_mean = good_result['papier_afstand'].mean()
    glas_mean = good_result['glas_afstand'].mean()
    plastic_mean = good_result['plastic_afstand'].mean()
    textiel_mean = good_result['textiel_afstand'].mean()
#     rest_mean, papier_mean, glas_mean, plastic_mean, textiel_mean
    score = 0.35 * rest_mean + 0.25 * plastic_mean + 0.2 * papier_mean + 0.15 * glas_mean + 0.05 * textiel_mean
    return score

In [25]:
avg_distance = calculate_weighted_distance(good_result)

### Calculate Penalties for every configuration
As discussed in the paper, there are certain policies regarding the placement of containers and the amount of households per container. The exact policies and their origin can be found in the report

In [26]:
def create_aanlsuitingen(good_result, total_join):
    """
    Function that returns dataframe aansluitingen that calculates amount of
    households per cluster and the percentage
    """
    aansluitingen = pd.DataFrame(good_result['poi_rest'].value_counts()).\
                join(pd.DataFrame(good_result['poi_papier'].value_counts()), how='outer').\
                join(pd.DataFrame(good_result['poi_plastic'].value_counts()), how='outer').\
                join(pd.DataFrame(good_result['poi_glas'].value_counts()), how='outer').\
                join(pd.DataFrame(good_result['poi_textiel'].value_counts()), how='outer')
    
    tmp_for_join = total_join[['van_s1_afv_nodes', 'rest', 'papier', 'plastic', 'glas', 'textiel', 'totaal']].drop_duplicates().set_index('van_s1_afv_nodes')
    aansluitingen = aansluitingen.join(tmp_for_join, how='left')
    
    aansluitingen['rest_perc'] = aansluitingen['poi_rest'] / aansluitingen['rest']
    aansluitingen['plastic_perc'] = aansluitingen['poi_plastic'] / aansluitingen['plastic'] / 2
    aansluitingen['papier_perc'] = aansluitingen['poi_papier'] / aansluitingen['papier'] / 2
    aansluitingen['glas_perc'] = aansluitingen['poi_glas'] / aansluitingen['glas'] / 2
    aansluitingen['textiel_perc'] = aansluitingen['poi_textiel'] / aansluitingen['textiel'] / 7.5

    return aansluitingen


In [27]:
def calculate_penalties(good_result):
    penalty1 = good_result[good_result['rest_afstand'] > 100]
    penalty1_sum = (penalty1['rest_afstand'].sum() - 100 * penalty1.shape[0])/good_result.shape[0] * 0.35
    penalty2 = good_result[good_result['plastic_afstand'] > 150]
    penalty2_sum = (penalty2['plastic_afstand'].sum() - 150 * penalty2.shape[0])/good_result.shape[0] * 0.25
    penalty3 = good_result[good_result['papier_afstand'] > 150]
    penalty3_sum = (penalty3['papier_afstand'].sum() - 150 * penalty3.shape[0])/good_result.shape[0] * 0.2
    penalty4 = good_result[good_result['glas_afstand'] > 150]
    penalty4_sum = (penalty4['glas_afstand'].sum() - 150 * penalty4.shape[0])/good_result.shape[0] * 0.15
    penalty5 = good_result[good_result['textiel_afstand'] > 300]
    penalty5_sum = (penalty5['textiel_afstand'].sum() - 300 * penalty5.shape[0])/good_result.shape[0] * 0.05

    penalty6 = aansluitingen[aansluitingen['rest_perc'] > 100]
    penalty6_sum = (penalty6['poi_rest'] - (penalty6['rest'] * 100)).sum()/ good_result.shape[0] * 0.35 * 1000
    penalty7 = aansluitingen[aansluitingen['plastic_perc'] > 100]
    penalty7_sum = (penalty7['poi_plastic'] - (penalty7['plastic'] * 200)).sum()/ good_result.shape[0] * 0.25 * 1000
    penalty8 = aansluitingen[aansluitingen['papier_perc'] > 100]
    penalty8_sum = (penalty8['poi_papier'] - (penalty8['papier'] * 200)).sum()/ good_result.shape[0] * 0.2 * 1000
    penalty9 = aansluitingen[aansluitingen['glas_perc'] > 100]
    penalty9_sum = (penalty9['poi_glas'] - (penalty9['glas'] * 200)).sum()/ good_result.shape[0] * 0.15 * 1000
    penalty10 = aansluitingen[aansluitingen['textiel_perc'] > 100]
    penalty10_sum = (penalty10['poi_textiel'] - (penalty10['textiel'] * 750)).sum()/ good_result.shape[0] * 0.05 * 1000

    total_penalties = sum([penalty1_sum, penalty2_sum, penalty3_sum, penalty4_sum, penalty5_sum,\
                           penalty6_sum, penalty7_sum, penalty8_sum, penalty9_sum, penalty10_sum])
    return total_penalties

In [28]:
aansluitingen = create_aanlsuitingen(good_result, total_join)
penalties = calculate_penalties(good_result)

### Final calculation
The average weighted walking distance and penalties are summed to come up with the total score for this configuration.

In [29]:
score = avg_distance + penalties
score

301.1170557429302